# AIG230 NLP (Week 3 Lab) — Notebook 2: Statistical Language Models (Train, Test, Evaluate)

This notebook focuses on **n-gram Statistical Language Models (SLMs)**:
- Train **unigram**, **bigram**, **trigram** models
- Handle **OOV** with `<UNK>`
- Apply **smoothing** (Add-k)
- Evaluate with **cross-entropy** and **perplexity**
- Do **next-word prediction** and simple **text generation**

> Industry framing: even if modern systems use neural LMs, n-gram LMs are still useful for
baselines, constrained domains, and for understanding evaluation.


### What is smoothing?

Smoothing is a way to stop a language model from saying “this can never happen.”

When we train a language model from data, it only knows what it has seen before.
If it never saw a particular word sequence, the model would normally give it a probability of zero.

Smoothing fixes that.
### Why is this a problem without smoothing?

Imagine the model learned English only by reading a small number of news articles.

If it never saw:

- “oil prices explode”

the model would conclude:

- “That sentence is impossible.”

But as humans, we know it could happen. The model just hasn’t seen it yet.

Without smoothing:

- One unseen word makes the whole sentence probability zero

- Evaluation breaks

- The model is too confident and too brittle

## 0) Setup


In [1]:
%pip install numpy pandas matplotlib


Note: you may need to restart the kernel to use updated packages.


In [2]:

import re
import math
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Dict


## 1) Data: domain text you might see in real systems


We use short texts that resemble:
- release notes
- incident summaries
- operational runbooks
- customer support messaging

In practice, you would load thousands to millions of lines.


In [3]:

corpus = [
    "vpn disconnects frequently after windows update",
    "password reset link expired user cannot login",
    "api requests timeout when latency spikes",
    "portal returns 500 error after deployment",
    "email delivery delayed messages queued",
    "mfa prompt never arrives user stuck at login",
    "wifi drops in meeting rooms access point reboot helps",
    "outlook search not returning results index corrupted",
    "printer driver install fails with error 1603",
    "teams calls choppy audio jitter high",
    "permission denied accessing shared drive though in correct group",
    "battery drains fast after bios update power settings unchanged",
    "push notifications not working on android app",
    "mailbox full cannot receive emails auto archive not running",
]

# Train/test split at sentence level
random.seed(42)
random.shuffle(corpus)
split = int(0.75 * len(corpus))
train_texts = corpus[:split]
test_texts = corpus[split:]

len(train_texts), len(test_texts), train_texts[:2], test_texts[:2]


(10,
 4,
 ['printer driver install fails with error 1603',
  'push notifications not working on android app'],
 ['email delivery delayed messages queued',
  'vpn disconnects frequently after windows update'])

## 2) Tokenization + special tokens


We will:
- lowercase
- keep alphanumerics
- split on whitespace
- add sentence boundary tokens: `<s>` and `</s>`

We will also map rare tokens to `<UNK>` based on training frequency.


In [4]:

def tokenize(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

def add_boundaries(tokens: List[str], n: int) -> List[str]:
    # For n-grams, prepend (n-1) start tokens for simpler context handling
    return ["<s>"]*(n-1) + tokens + ["</s>"]

# Example
tokens = tokenize("Printer driver install fails with error 1603")
add_boundaries(tokens, n=3)


['<s>',
 '<s>',
 'printer',
 'driver',
 'install',
 'fails',
 'with',
 'error',
 '1603',
 '</s>']

## 3) Build vocabulary and handle OOV with <UNK>


In [5]:

# Build vocab from training data
train_tokens_flat = []
for t in train_texts:
    train_tokens_flat.extend(tokenize(t))

freq = Counter(train_tokens_flat)

# Typical practical rule: map tokens with frequency <= 1 to <UNK> in small corpora
min_count = 2
vocab = {w for w, c in freq.items() if c >= min_count}
vocab |= {"<UNK>", "<s>", "</s>"}

def replace_oov(tokens: List[str], vocab: set) -> List[str]:
    return [tok if tok in vocab else "<UNK>" for tok in tokens]

# Show OOV effect
sample = tokenize(test_texts[0])
sample, replace_oov(sample, vocab)


(['email', 'delivery', 'delayed', 'messages', 'queued'],
 ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>'])

## 4) Train n-gram counts (unigram, bigram, trigram)


We will compute:
- `ngram_counts[(w1,...,wn)]`
- `context_counts[(w1,...,w_{n-1})]`

Then probability:
\ndefault:  P(w_n | context) = count(context + w_n) / count(context)

This fails when an n-gram is unseen, so we add smoothing.


In [6]:
def count_ngrams(texts: List[str], n: int, vocab: set) -> Tuple[Counter, Counter]:
    ngram_counts = Counter()
    context_counts = Counter()
    for text in texts:
        tokens = replace_oov(tokenize(text), vocab)
        tokens = add_boundaries(tokens, n)
        for i in range(len(tokens)-n+1):
            ngram = tuple(tokens[i : i+n])
            context = ngram[:-1]
            ngram_counts[ngram] += 1
            context_counts[context] += 1
    return ngram_counts, context_counts

# Train Unigram, Bigram, Trigram
uni_counts, uni_ctx = count_ngrams(train_texts, 1, vocab)
bi_counts, bi_ctx   = count_ngrams(train_texts, 2, vocab)
tri_counts, tri_ctx = count_ngrams(train_texts, 3, vocab)

print("Unigrams:", len(uni_counts))
print("Bigrams:", len(bi_counts))
print("Trigrams:", len(tri_counts))
print("Most common bigrams:", bi_counts.most_common(5))

Unigrams: 5
Bigrams: 10
Trigrams: 17
Most common bigrams: [(('<UNK>', '<UNK>'), 51), (('<s>', '<UNK>'), 10), (('<UNK>', '</s>'), 10), (('<UNK>', 'not'), 3), (('not', '<UNK>'), 3)]


## 5) Add-k smoothing and probability function


### What does Add-k smoothing do?
Add-k smoothing tells the model:

- “Even if you didn’t see something, assume it could still happen a little bit.”

It does this by:

- Giving every possible next word a tiny amount of probability

- Not just the ones seen in training

So instead of:

- seen → possible

- unseen → impossible

We get:

- seen → more likely

- unseen → less likely, but still possible


### Why is it called Add-k?

Because we add a small number k to every word count.

Think of it as:

- adding a tiny “imaginary observation” for every word

- so no word ever has zero probability

When k is small (like 0.1 or 0.5), it gently smooths the probabilities instead of overpowering real data.

In [7]:
# This function calculates the probability of a word appearing next, given the previous words, while making sure the probability is never zero.
def prob_addk(ngram: tuple, ngram_counts: Counter, context_counts: Counter, V: int, k: float = 1.0) -> float:
    context = ngram[:-1]
    count_ngram = ngram_counts[ngram]
    count_context = context_counts[context]
    # P = (count(w_n | context) + k) / (count(context) + k*V)
    return (count_ngram + k) / (count_context + k * V)

# Test it
V = len(vocab)
print("P('update' | 'windows') with k=1:", 
      prob_addk(('windows', 'update'), bi_counts, bi_ctx, V, k=1.0))
print("P('<UNK>' | 'windows') with k=1:", 
      prob_addk(('windows', '<UNK>'), bi_counts, bi_ctx, V, k=1.0))

P('update' | 'windows') with k=1: 0.16666666666666666
P('<UNK>' | 'windows') with k=1: 0.16666666666666666


## 6) Evaluate: cross-entropy and perplexity on test set


We evaluate an LM by how well it predicts held-out text.

Cross-entropy (average negative log probability):
H = - (1/N) * sum log2 P(w_i | context)

Perplexity:
PP = 2^H

Lower perplexity is better.


In [8]:
def get_perplexity(texts: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k: float = 1.0) -> float:
    log_prob_sum = 0.0
    N = 0
    V = len(vocab)
    for text in texts:
        tokens = replace_oov(tokenize(text), vocab)
        tokens = add_boundaries(tokens, n)
        for i in range(len(tokens)-n+1):
            ngram = tuple(tokens[i : i+n])
            p = prob_addk(ngram, ngram_counts, context_counts, V, k)
            log_prob_sum += math.log2(p)
            N += 1
    cross_entropy = -log_prob_sum / N
    return 2 ** cross_entropy

print(f"Unigram Perplexity: {get_perplexity(test_texts, 1, uni_counts, uni_ctx, vocab):.4f}")
print(f"Bigram Perplexity:  {get_perplexity(test_texts, 2, bi_counts, bi_ctx, vocab):.4f}")
print(f"Trigram Perplexity: {get_perplexity(test_texts, 3, tri_counts, tri_ctx, vocab):.4f}")

Unigram Perplexity: 1.8515
Bigram Perplexity:  1.9543
Trigram Perplexity: 2.0941


## 7) Next-word prediction (top-k)


Given a context, compute the probability of each candidate next token and return the top-k.

This mirrors:
- autocomplete in constrained domains
- template suggestion systems
- command prediction in runbooks


In [9]:
def predict_next(context_str: str, n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k: float = 1.0, top: int = 5):
    tokens = tokenize(context_str)
    # Grab last n-1 tokens
    context_tokens = tokens[-(n-1):] if n > 1 else []
    # Apply OOV mapping
    context_tokens = replace_oov(context_tokens, vocab)
    context_tuple = tuple(context_tokens)
    
    candidates = []
    V = len(vocab)
    # Iterate all possible next words
    for w in vocab:
        if w == "<s>": continue # Cannot predict start token
        ngram = context_tuple + (w,)
        p = prob_addk(ngram, ngram_counts, context_counts, V, k)
        candidates.append((w, p))
    
    candidates.sort(key=lambda x: -x[1])
    return candidates[:top]

# Example predictions
print("Next word after 'vpn':", predict_next("vpn", 2, bi_counts, bi_ctx, vocab))
print("Next word after 'cannot':", predict_next("cannot", 2, bi_counts, bi_ctx, vocab))

Next word after 'vpn': [('<UNK>', 0.7123287671232876), ('</s>', 0.1506849315068493), ('not', 0.0547945205479452), ('error', 0.0410958904109589), ('after', 0.0273972602739726)]
Next word after 'cannot': [('<UNK>', 0.7123287671232876), ('</s>', 0.1506849315068493), ('not', 0.0547945205479452), ('error', 0.0410958904109589), ('after', 0.0273972602739726)]


## 8) Simple generation (bigram or trigram)


Text generation is not the main goal in SLMs, but it helps you verify:
- boundary handling
- smoothing
- OOV decisions

We will sample tokens until we hit `</s>`.


In [10]:

def sample_next(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5):
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    words = [w for w in vocab if w != "<s>"]
    probs = []
    for w in words:
        ng = context + (w,)
        probs.append(prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth))
    # Normalize
    s = sum(probs)
    probs = [p/s for p in probs]
    return random.choices(words, weights=probs, k=1)[0]

def generate(n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, max_len: int = 20, k_smooth: float = 0.5):
    tokens = ["<s>"]*(n-1) if n > 1 else []
    out = []
    for _ in range(max_len):
        w = sample_next(tokens, n, ngram_counts, context_counts, vocab, k_smooth=k_smooth)
        if w == "</s>":
            break
        out.append(w)
        tokens.append(w)
    return " ".join(out)

for _ in range(5):
    print("BIGRAM:", generate(2, bi_counts, bi_ctx, vocab, max_len=18))


BIGRAM: error error <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
BIGRAM: <UNK> <UNK> <UNK> <UNK>
BIGRAM: <UNK> <UNK> <UNK> <UNK>
BIGRAM: <UNK>
BIGRAM: not error <UNK> <UNK> <UNK> <UNK> <UNK>


## 9) Model comparison: effect of n and smoothing


Try different `k` values. Notes:
- `k=1.0` is Laplace smoothing (often too strong)
- smaller `k` (like 0.1 to 0.5) is often better

In real corpora, trigrams often beat bigrams, but require more data.


In [11]:
ks = [0.01, 0.1, 0.5, 1.0, 2.0]
print(f"{'-'*40}")
print(f"{'k':<6} {'Bigram PP':<12} {'Trigram PP':<12}")
print(f"{'-'*40}")

for k in ks:
    pp_bi = get_perplexity(test_texts, 2, bi_counts, bi_ctx, vocab, k=k)
    pp_tri = get_perplexity(test_texts, 3, tri_counts, tri_ctx, vocab, k=k)
    print(f"{k:<6} {pp_bi:<12.4f} {pp_tri:<12.4f}")

----------------------------------------
k      Bigram PP    Trigram PP  
----------------------------------------
0.01   1.7639       1.7523      
0.1    1.7875       1.8020      
0.5    1.8712       1.9553      
1.0    1.9543       2.0941      
2.0    2.0911       2.3098      


## Exercises (do these during lab)
1) Add 20 more realistic domain sentences to the corpus and re-run training/evaluation.  
2) Change `min_count` (OOV threshold) and explain how perplexity changes.  
3) Implement **backoff**: if a trigram is unseen, fall back to bigram; if unseen, fall back to unigram.
